# Load Conf and Credentials

## Load Directory Locations

In [9]:
import json
import os

# Check if the file exists and load the JSON file into a dictionary
file_path = r'C:\Users\mike\Develop\Projects\Code Notebook\Credentials\locations_conf.json'
if os.path.exists(file_path):
    with open(file_path, 'r') as f:
        locations_data = json.load(f)
    print(locations_data)
else:
    print(f"File not found: {file_path}")

{'Common_Funcs_Dir': '/Users/mike/Develop/Projects/Code Notebook/Common/Functions', 'Credentials_Dir': '/Users/mike/Develop/Projects/Code Notebook/Credentials', 'Rel_Pickes_Dir': '../.pickles', 'Pub_Data_Dir': "'/Users/mike/Data/Public"}


### Get the Common Funcs Dir into the Sys Path
This appears to be required bc the Funcs are .py files vs .ipynb files

In [10]:
import sys
sys.path.append(locations_data['Common_Funcs_Dir'])
from func_Load_Data_to_Frame import *

# Load The Source Data to a DF

In [11]:
import pandas as pd

source_file = 'LAPD_Crime_Data.json.zip'
source_path = os.path.join(locations_data['Pub_Data_Dir'].strip("'"), source_file)
df = pd.read_json(source_path, compression='zip')

# Evaluate the Schmema Properties

## Published Schema:
https://data.lacity.org/Public-Safety/Crime-Data-from-2020-to-Present/2nrs-mtv8/about_data

## Generated Schema in the df

In [12]:
# Show schema
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 585000 entries, 0 to 584999
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   dr_no           585000 non-null  int64  
 1   date_rptd       585000 non-null  object 
 2   date_occ        585000 non-null  object 
 3   time_occ        585000 non-null  int64  
 4   area            585000 non-null  int64  
 5   area_name       585000 non-null  object 
 6   rpt_dist_no     585000 non-null  int64  
 7   part_1_2        585000 non-null  int64  
 8   crm_cd          585000 non-null  int64  
 9   crm_cd_desc     585000 non-null  object 
 10  mocodes         491719 non-null  object 
 11  vict_age        585000 non-null  int64  
 12  vict_sex        496186 non-null  object 
 13  vict_descent    496179 non-null  object 
 14  premis_cd       584990 non-null  float64
 15  premis_desc     584585 non-null  object 
 16  weapon_used_cd  175921 non-null  float64
 17  weapon_des

## Now Align the Data Types using the Published Schema @
https://data.lacity.org/Public-Safety/Crime-Data-from-2020-to-Present/2nrs-mtv8/about_data

### Convert the DR # (Div of Records Num)

In [13]:
# Convert dr_no to a formatted string and update in place
df['dr_no'] = df['dr_no'].apply(lambda x: f"{str(x)[:2]}-{str(x)[2:4]}-{str(x)[4:]}")

# Display the type and head of the dr_no column
print(df['dr_no'].dtype)
print(df['dr_no'].head())

object
0    22-05-06019
1    22-08-05315
2    22-14-05638
3    22-21-05108
4    22-12-05693
Name: dr_no, dtype: object


### Convert the Date Reported to a Date in format YYYY-MM-DD

In [14]:
# Convert date columns to datetime
df['date_rptd'] = pd.to_datetime(df['date_rptd'])
df['date_occ'] = pd.to_datetime(df['date_occ'])

# Display the updated DataFrame with the new formatted date_rptd and date_occ columns
print(df[['date_rptd', 'date_occ']].head())
print(df[['date_rptd', 'date_occ']].dtypes)

   date_rptd   date_occ
0 2022-02-01 2020-02-01
1 2022-02-01 2020-01-01
2 2022-02-01 2020-09-01
3 2022-02-01 2021-11-29
4 2022-02-01 2021-12-21
date_rptd    datetime64[ns]
date_occ     datetime64[ns]
dtype: object


### Combine the Date Occurred and Time Occ into a single Timestamp and remove the orinal Time Occ col

In [15]:
# Combine date_occ and time_occ into a single datetime column
df['datetime_occ'] = pd.to_datetime(df['date_occ'].dt.strftime('%Y-%m-%d') + ' ' + df['time_occ'].astype(str).str.zfill(4).str[:2] + ':' + df['time_occ'].astype(str).str.zfill(4).str[2:] + ':00', format='%Y-%m-%d %H:%M:%S')

# Drop the original time_occ and date_occ columns
df.drop(columns=['time_occ', 'date_occ'], inplace=True)

# Reorder columns to place datetime_occ in the original position of time_occ
cols = df.columns.tolist()
time_occ_index = cols.index('datetime_occ')
cols.insert(3, cols.pop(time_occ_index))
df = df[cols]

# Display the updated DataFrame with the modified datetime_occ column
print(df[['datetime_occ']].head())
# Display the data type of the datetime_occ column
print(df['datetime_occ'].dtypes)

         datetime_occ
0 2020-02-01 12:00:00
1 2020-01-01 12:00:00
2 2020-09-01 14:25:00
3 2021-11-29 12:00:00
4 2021-12-21 18:00:00
datetime64[ns]


### Convert the Category Type Cols to Category types for efficiency

In [16]:
# Convert categorical columns to category dtype
categorical_columns = ['area_name', 'crm_cd_desc', 'mocodes', 'vict_sex', 'vict_descent', 'premis_desc', 'weapon_desc', 'status', 'status_desc', 'location', 'cross_street']
for col in categorical_columns:
    df[col] = df[col].astype('category')


### Convert the Various Codes to Short Ints for space

In [21]:
# Recheck the integer columns as some may have changed and convert to a short int
integer_columns = ['area', 'rpt_dist_no', 'part_1_2', 'crm_cd', 'vict_age', 'premis_cd', 'weapon_used_cd', 'crm_cd_1', 'crm_cd_2', 'crm_cd_3', 'crm_cd_4']
for col in integer_columns:
    if col in df.columns:
        df[col] = df[col].fillna(0).astype('int16')

# Display the columns and their data types
print(df[integer_columns].dtypes)

area              int16
rpt_dist_no       int16
part_1_2          int16
crm_cd            int16
vict_age          int16
premis_cd         int16
weapon_used_cd    int16
crm_cd_1          int16
crm_cd_2          int16
crm_cd_3          int16
crm_cd_4          int16
dtype: object


### Convert the Lat and Lon Cols to Floats

In [23]:
# Convert latitude and longitude to float64
df['lat'] = df['lat'].astype('float64')
df['lon'] = df['lon'].astype('float64')

# Verify changes
print(df[['lat', 'lon']].dtypes)

lat    float64
lon    float64
dtype: object


# Normalize the df into Dims and Facts
See Schema here:
https://lucid.app/lucidspark/a6f7a7bf-63d8-4aa7-ac61-6caa235ac916/edit?invitationId=inv_43e7079e-9d82-4f94-8907-0b93e812a852

## Dimension Areas

In [24]:
# Create dimension tables
dim_area = (
    df[['area', 'area_name']]
    .drop_duplicates()
    .reset_index(drop=True)
    .rename(columns={'area': 'fk_area'})
)
# Show datatypes of the dimension table
print(dim_area.dtypes)

# Show some sample data from the dimension table
print(dim_area.head())

fk_area         int16
area_name    category
dtype: object
   fk_area    area_name
0        5       Harbor
1        8      West LA
2       14      Pacific
3       21      Topanga
4       12  77th Street


## Dimension Crimes

In [25]:
dim_crime = (
    df[['crm_cd', 'crm_cd_desc']]
    .drop_duplicates()
    .reset_index(drop=True)
    .rename(columns={'crm_cd': 'fk_crm_cd'})
)
# Display a few rows of the dimension table
print(dim_crime.head())

   fk_crm_cd                                      crm_cd_desc
0        627          CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT
1        236            INTIMATE PARTNER - AGGRAVATED ASSAULT
2        354                                THEFT OF IDENTITY
3        649                 DOCUMENT FORGERY / STOLEN FELONY
4        420  THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)


## Dimension Victim Sex

In [26]:
dim_victim_sex = (
    df[['vict_sex']]
    .drop_duplicates()
    .reset_index(drop=True)
)

# Set any missing data to 'X'
dim_victim_sex['vict_sex'] = dim_victim_sex['vict_sex'].fillna('X')

# Add sex_desc column
dim_victim_sex['sex_desc'] = dim_victim_sex['vict_sex'].map({
    'M': 'Male',
    'F': 'Female',
    'X': 'UNKNOWN',
    'H': 'UNKNOWN'
})

# Drop duplicates
dim_victim_sex = dim_victim_sex.drop_duplicates()

# Drop any rows with NaN values
dim_victim_sex = dim_victim_sex.dropna()

# Show datatypes of the dimension table
print(dim_victim_sex.dtypes)
print(dim_victim_sex.head())

vict_sex    category
sex_desc      object
dtype: object
  vict_sex sex_desc
0        M     Male
1        F   Female
2        X  UNKNOWN
4        H  UNKNOWN


## Dimension Victim Descent

In [27]:
dim_victim = (
    df[['vict_descent']]
    .drop_duplicates()
    .reset_index(drop=True)
)

# Set any missing data to 'X'
dim_victim['vict_descent'] = dim_victim['vict_descent'].fillna('X')

# Add descent_desc column
dim_victim['descent_desc'] = dim_victim['vict_descent'].map({
    'A': 'Other Asian',
    'B': 'Black',
    'C': 'Chinese',
    'D': 'Cambodian',
    'F': 'Filipino',
    'G': 'Guamanian',
    'H': 'Hispanic/Latin/Mexican',
    'I': 'American Indian/Alaskan Native',
    'J': 'Japanese',
    'K': 'Korean',
    'L': 'Laotian',
    'O': 'Other',
    'P': 'Pacific Islander',
    'S': 'Samoan',
    'U': 'Hawaiian',
    'V': 'Vietnamese',
    'W': 'White',
    'X': 'Unknown',
    'Z': 'Asian Indian'
})

# Drop duplicates
dim_victim = dim_victim.drop_duplicates()

# Drop any rows with NaN values
dim_victim = dim_victim.dropna()

# Show datatypes of the dimension table
print(dim_victim.dtypes)
print(dim_victim.head(20))

vict_descent    category
descent_desc      object
dtype: object
   vict_descent                    descent_desc
0             H          Hispanic/Latin/Mexican
1             W                           White
2             X                         Unknown
4             B                           Black
5             O                           Other
6             A                     Other Asian
7             F                        Filipino
8             C                         Chinese
9             V                      Vietnamese
10            K                          Korean
11            J                        Japanese
12            S                          Samoan
13            I  American Indian/Alaskan Native
14            P                Pacific Islander
15            Z                    Asian Indian
16            L                         Laotian
17            G                       Guamanian
18            D                       Cambodian
19            U         

## Dim Premises

In [28]:
dim_premise = (
    df[['premis_cd', 'premis_desc']]
    .drop_duplicates()
    .reset_index(drop=True)
    .rename(columns={'premis_cd': 'fk_premis_cd'})
)
# Show datatypes of the dimension table
print(dim_premise.dtypes)

# Show some sample data from the dimension table
print(dim_premise.head())

fk_premis_cd       int16
premis_desc     category
dtype: object
   fk_premis_cd                                   premis_desc
0           501                        SINGLE FAMILY DWELLING
1           502  MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)
2           222                                    LAUNDROMAT
3           101                                        STREET
4           108                                   PARKING LOT


## Dim Weapons

In [29]:
dim_weapon = (
    df[['weapon_used_cd', 'weapon_desc']]
    .drop_duplicates()
    .reset_index(drop=True)
    .rename(columns={'weapon_used_cd': 'fk_weapon_used_cd'})
)
# Show datatypes of the dimension table
print(dim_weapon.dtypes)

# Show some sample data from the dimension table
print(dim_weapon.head())

fk_weapon_used_cd       int16
weapon_desc          category
dtype: object
   fk_weapon_used_cd                                     weapon_desc
0                400  STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)
1                  0                                             NaN
2                503                         CAUSTIC CHEMICAL/POISON
3                500                     UNKNOWN WEAPON/OTHER WEAPON
4                205                                   KITCHEN KNIFE


## Dim Status

In [30]:
dim_status = (
    df[['status', 'status_desc']]
    .drop_duplicates()
    .reset_index(drop=True)
    .rename(columns={'status': 'fk_status'})
)
# Show datatypes of the dimension table
print(dim_status.dtypes)
print(dim_status.head())

fk_status      category
status_desc    category
dtype: object
  fk_status   status_desc
0        AO   Adult Other
1        IC   Invest Cont
2        AA  Adult Arrest
3        JA    Juv Arrest
4        JO     Juv Other


## Dim Locations

In [31]:
dim_location = (
    df[['lat', 'lon']]
    .drop_duplicates()
    .reset_index(drop=True)
)

# Add additional columns based on Schema at https://geocode.maps.co/reverse
# This data will be added downstream in the ETL process
dim_location['geo_place_id'] = None
dim_location['geo_osm_type'] = None
dim_location['geo_osm_id'] = None
dim_location['geo_display_name'] = None
dim_location['geo_road'] = None
dim_location['geo_neighbourhood'] = None
dim_location['geo_suburb'] = None
dim_location['geo_city'] = None
dim_location['geo_state'] = None
dim_location['geo_ISO3166-2-lvl4'] = None
dim_location['geo_postcode'] = None
dim_location['geo_country'] = None
dim_location['geo_country_code'] = None
dim_location['geo_boundingbox'] = None
# Show datatypes of the dimension table
print(dim_location.dtypes)

# Show some sample data from the dimension table
print(dim_location.head())

lat                   float64
lon                   float64
geo_place_id           object
geo_osm_type           object
geo_osm_id             object
geo_display_name       object
geo_road               object
geo_neighbourhood      object
geo_suburb             object
geo_city               object
geo_state              object
geo_ISO3166-2-lvl4     object
geo_postcode           object
geo_country            object
geo_country_code       object
geo_boundingbox        object
dtype: object
       lat       lon geo_place_id geo_osm_type geo_osm_id geo_display_name  \
0  33.8201 -118.3015         None         None       None             None   
1  34.0326 -118.3941         None         None       None             None   
2  33.9875 -118.4668         None         None       None             None   
3  34.1707 -118.6565         None         None       None             None   
4  33.9638 -118.2629         None         None       None             None   

  geo_road geo_neighbourhood geo_subu

## Create the Fact Table

In [32]:
# Create fact table
crime_facts = df[
    [
        # Unique Facts
        'dr_no', 'date_rptd', 'datetime_occ', 'rpt_dist_no', 'vict_age', 
        # Dim Locations
        'lat', 'lon', 
        # Dim Area
        'area', 
        # Dim Premise
        'premis_cd',
        # Dim Crime
        'crm_cd', 
        # Dim Victim Sex
        'vict_sex', 
        # Dim Victim Descent
        'vict_descent',
        # Dim Weapon
        'weapon_used_cd',
        # Dim Status 
        'status'
    ]
].copy()

# Handle missing data
crime_facts['vict_sex'] = crime_facts['vict_sex'].cat.add_categories(['Unknown']).fillna('Unknown')
crime_facts['vict_descent'] = crime_facts['vict_descent'].cat.add_categories(['Unknown']).fillna('Unknown')
crime_facts['status'] = crime_facts['status'].cat.add_categories(['Unknown']).fillna('Unknown')

# Display schema and types
print(crime_facts.info())

# Display fact table
display(crime_facts.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 585000 entries, 0 to 584999
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   dr_no           585000 non-null  object        
 1   date_rptd       585000 non-null  datetime64[ns]
 2   datetime_occ    585000 non-null  datetime64[ns]
 3   rpt_dist_no     585000 non-null  int16         
 4   vict_age        585000 non-null  int16         
 5   lat             585000 non-null  float64       
 6   lon             585000 non-null  float64       
 7   area            585000 non-null  int16         
 8   premis_cd       585000 non-null  int16         
 9   crm_cd          585000 non-null  int16         
 10  vict_sex        585000 non-null  category      
 11  vict_descent    585000 non-null  category      
 12  weapon_used_cd  585000 non-null  int16         
 13  status          585000 non-null  category      
dtypes: category(3), datetime64[ns](2), f

,dr_no,date_rptd,datetime_occ,rpt_dist_no,vict_age,lat,lon,area,premis_cd,crm_cd,vict_sex,vict_descent,weapon_used_cd,status
0,22-05-06019,2022-02-01,2020-02-01 12:00:00,507,12,33.8201,-118.3015,5,501,627,M,H,400,AO
1,22-08-05315,2022-02-01,2020-01-01 12:00:00,898,31,34.0326,-118.3941,8,502,236,F,H,400,IC
2,22-14-05638,2022-02-01,2020-09-01 14:25:00,1431,42,33.9875,-118.4668,14,501,354,M,W,0,IC
3,22-21-05108,2022-02-01,2021-11-29 12:00:00,2161,89,34.1707,-118.6565,21,501,649,F,W,0,IC
4,22-12-05693,2022-02-01,2021-12-21 18:00:00,1269,35,33.9638,-118.2629,12,222,236,F,H,400,AO


# Downstream Post Processes - Retrieve Geo Code into Location Dim

## Load the Location Dim from the Pickle

In [33]:
import pandas as pd
import os
import zipfile

# Set the pickle file path
pickle_file = 'dim_location.pkl'
pickle_path = os.path.join(locations_data['Rel_Pickes_Dir'], pickle_file)

# Check if the pickle file is zipped
if os.path.exists(pickle_path + '.zip'):
	with zipfile.ZipFile(pickle_path + '.zip', 'r') as z:
		with z.open(pickle_file) as f:
			dim_location = pd.read_pickle(f)
else:
	# Load the dimension table from the pickle file
	try:
		dim_location = pd.read_pickle(pickle_path)
	except FileNotFoundError:
		print("Pickle file not found. Please ensure the file exists.")

# Display the loaded DataFrame
print(dim_location.head())

        lat       lon geo_place_id geo_osm_type geo_osm_id  \
0   33.8201 -118.3015    281393593          way   13356241   
6   34.0326 -118.3941    284925955          way  165791832   
7   33.9875 -118.4668    281444680          way  168954633   
11  34.1707 -118.6565    285003597          way  402526611   
12  33.9638 -118.2629    281472844          way  165899885   

                                     geo_display_name               geo_road  \
0   Halldale Avenue, Los Angeles, California, 9050...        Halldale Avenue   
6   South Canfield Avenue, Castle Heights, Los Ang...  South Canfield Avenue   
7   Venice Way, Venice Canal Historic District, Ve...             Venice Way   
11  Burbank Boulevard, Los Angeles, Los Angeles Co...      Burbank Boulevard   
12  Stanford Avenue, Florence, Los Angeles, Los An...        Stanford Avenue   

                 geo_neighbourhood geo_suburb     geo_city   geo_state  \
0                             None       None  Los Angeles  California  

## Find the Locations that need to be Processed

In [34]:
tmp_locs_to_be_processed = dim_location[dim_location['geo_place_id'].isnull()][['lat', 'lon']].drop_duplicates()
print(f"Number of records in the df: {len(tmp_locs_to_be_processed)}")

Number of records in the df: 0


## Get the API Credentials

In [35]:
import json

# Set the path for the JSON file
credentials_dir = locations_data['Credentials_Dir']
json_file_path = os.path.join(credentials_dir, 'geocode_api_key.json')

# Read the credentials from the JSON file
with open(json_file_path, 'r') as f:
    api_key_data = json.load(f)

geo_api_credentials = {
    'API_Key': api_key_data['API_Key'],
    'Reverse_URL': api_key_data['Reverse_URL'],
    'Max_Lookups': api_key_data['Max_Lookups'],
    'Rate_Limit_per_Sec': api_key_data['Rate_Limit_per_Sec']
}

# Mask the API key for security
masked_key = geo_api_credentials['API_Key'][:5] + 'xxxxx'
geo_api_credentials['API_Key'] = masked_key

print(geo_api_credentials)


{'API_Key': '679e7xxxxx', 'Reverse_URL': 'https://geocode.maps.co/reverse', 'Max_Lookups': 50000, 'Rate_Limit_per_Sec': 1}


## Use the GeoCode API to Fetch Location Details using the Reverse Lookup from the Lat/Lon
See https://geocode.maps.co/

### Write a Func for the Fetch
This will be moved to Common Funcs Later

In [36]:
import requests
from tqdm import tqdm
import time

def Fetch_Geo_Data(api_credentials, locations):
    base_url = api_credentials['Reverse_URL']
    api_key = api_credentials['API_Key']
    max_lookups = api_credentials['Max_Lookups']
    rate_limit_per_sec = api_credentials['Rate_Limit_per_Sec']
    
    geo_data = []
    
    for index, row in tqdm(locations.iterrows(), total=locations.shape[0], desc="Fetching Geo Data"):
        if len(geo_data) >= max_lookups:
            print("Reached the maximum number of lookups for the day.")
            break
        
        params = {
            'lat': row['lat'],
            'lon': row['lon'],
            'api_key': api_key
        }
        
        attempts = 0
        while attempts < 3:
            try:
                response = requests.get(base_url, params=params)
                url = f"{base_url}?lat={row['lat']}&lon={row['lon']}&api_key={api_key}"
                response = requests.get(url)
                if response.status_code == 200:
                    if 'place_id' not in response.json():
                        print(f"Response content for lat: {row['lat']}, lon: {row['lon']} - {response.json()}")
                    geo_data.append(response.json())
                    break
                else:
                    attempts += 1
                    time.sleep(rate_limit_per_sec)
            except requests.exceptions.RequestException as e:
                print(f"Request failed: {e}")
                attempts += 1
                time.sleep(rate_limit_per_sec)
        
        if attempts == 3:
            print(f"Failed to fetch data for lat: {row['lat']}, lon: {row['lon']} after 3 attempts.")
            geo_data.append(None)
    
    return geo_data

### Now call the Func

In [37]:

# Example usage
geo_data = Fetch_Geo_Data(geo_api_credentials, tmp_locs_to_be_processed)
print(geo_data[:5])
# Update the dim_location DataFrame with the fetched geo data
for i, data in enumerate(geo_data):
    if data:
        dim_location.at[tmp_locs_to_be_processed.index[i], 'geo_place_id'] = data.get('place_id')
        dim_location.at[tmp_locs_to_be_processed.index[i], 'geo_osm_type'] = data.get('osm_type')
        dim_location.at[tmp_locs_to_be_processed.index[i], 'geo_osm_id'] = data.get('osm_id')
        dim_location.at[tmp_locs_to_be_processed.index[i], 'geo_display_name'] = data.get('display_name')
        dim_location.at[tmp_locs_to_be_processed.index[i], 'geo_road'] = data.get('address', {}).get('road')
        dim_location.at[tmp_locs_to_be_processed.index[i], 'geo_neighbourhood'] = data.get('address', {}).get('neighbourhood')
        dim_location.at[tmp_locs_to_be_processed.index[i], 'geo_suburb'] = data.get('address', {}).get('suburb')
        dim_location.at[tmp_locs_to_be_processed.index[i], 'geo_city'] = data.get('address', {}).get('city')
        dim_location.at[tmp_locs_to_be_processed.index[i], 'geo_state'] = data.get('address', {}).get('state')
        dim_location.at[tmp_locs_to_be_processed.index[i], 'geo_ISO3166-2-lvl4'] = data.get('address', {}).get('ISO3166-2-lvl4')
        dim_location.at[tmp_locs_to_be_processed.index[i], 'geo_postcode'] = data.get('address', {}).get('postcode')
        dim_location.at[tmp_locs_to_be_processed.index[i], 'geo_country'] = data.get('address', {}).get('country')
        dim_location.at[tmp_locs_to_be_processed.index[i], 'geo_country_code'] = data.get('address', {}).get('country_code')
        dim_location.at[tmp_locs_to_be_processed.index[i], 'geo_boundingbox'] = data.get('boundingbox')

# Display updated dim_location DataFrame
print(dim_location.head())

Fetching Geo Data: 0it [00:00, ?it/s]

[]
        lat       lon geo_place_id geo_osm_type geo_osm_id  \
0   33.8201 -118.3015    281393593          way   13356241   
6   34.0326 -118.3941    284925955          way  165791832   
7   33.9875 -118.4668    281444680          way  168954633   
11  34.1707 -118.6565    285003597          way  402526611   
12  33.9638 -118.2629    281472844          way  165899885   

                                     geo_display_name               geo_road  \
0   Halldale Avenue, Los Angeles, California, 9050...        Halldale Avenue   
6   South Canfield Avenue, Castle Heights, Los Ang...  South Canfield Avenue   
7   Venice Way, Venice Canal Historic District, Ve...             Venice Way   
11  Burbank Boulevard, Los Angeles, Los Angeles Co...      Burbank Boulevard   
12  Stanford Avenue, Florence, Los Angeles, Los An...        Stanford Avenue   

                 geo_neighbourhood geo_suburb     geo_city   geo_state  \
0                             None       None  Los Angeles  Californi

# Pickle & Zip Dims, Facts for Downstream Processing

In [38]:
import os

# Create the directory if it doesn't exist
pickles_dir = locations_data['Rel_Pickes_Dir']
os.makedirs(pickles_dir, exist_ok=True)

# Define a list of the dimension and fact tables along with their filenames
pickle_files = [
    (dim_area, 'dim_area.pkl'),
    (dim_crime, 'dim_crime.pkl'),
    (dim_victim, 'dim_victim.pkl'),
    (dim_premise, 'dim_premise.pkl'),
    (dim_weapon, 'dim_weapon.pkl'),
    (dim_status, 'dim_status.pkl'),
    (dim_location, 'dim_location.pkl'),
    (crime_facts, 'crime_facts.pkl')
]

# Use a while loop to pickle the tables
i = 0
while i < len(pickle_files):
    df, filename = pickle_files[i]
    pickle_path = os.path.join(pickles_dir, filename)
    df.to_pickle(pickle_path)
    
    # Zip the pickle file
    with zipfile.ZipFile(pickle_path + '.zip', 'w', zipfile.ZIP_DEFLATED) as z:
        z.write(pickle_path, filename)
    
    # Remove the uncompressed pickle file
    os.remove(pickle_path)
    
    i += 1